## Atelier 2 - Discrétisation de la variance de bloc

### 🎯 But pédagogique  
Étudier l’influence de la discrétisation spatiale sur la précision du calcul de la variance de blocs à partir d’un modèle de covariance.

### ⚙️ Impact de la discrétisation

Dans la pratique numérique, cette intégrale double est approchée par une discrétisation spatiale du bloc en un nombre fini de points. La précision du calcul de la variance de bloc dépend donc de la résolution choisie :  
- Une faible résolution (peu de points) entraîne une approximation grossière et une estimation moins précise de la variance.  
- Une résolution élevée (beaucoup de points) améliore la précision mais augmente le coût de calcul.  

Ce scénario permet d’étudier quantitativement cet impact, en observant comment la variance estimée varie avec la densité de points de discrétisation.




In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Covariances --- #
def spherical_covariance(h, range_, sill):
    cov = np.zeros_like(h)
    mask = h <= range_
    hr = h[mask] / range_
    cov[mask] = sill * (1 - 1.5 * hr + 0.5 * hr**3)
    return cov

def exponential_covariance(h, range_, sill):
    return sill * np.exp(-3 * h / range_)

def gaussian_covariance(h, range_, sill):
    return sill * np.exp(-3 * (h / range_)**2)

def get_covariance_model(name):
    return {
        'sphérique': spherical_covariance,
        'exponentiel': exponential_covariance,
        'gaussien': gaussian_covariance
    }[name]

# --- Distance anisotrope ---
def anisotropic_distance(dx, dy=0, dz=0, range_x=1, range_y=1, range_z=1, angle_deg=0):
    angle_rad = np.deg2rad(angle_deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)

    dx_rot = cos_a * dx + sin_a * dy
    dy_rot = -sin_a * dx + cos_a * dy
    dz_rot = dz  # Pas de rotation en Z

    return np.sqrt((dx_rot / range_x)**2 + (dy_rot / range_y)**2 + (dz_rot / range_z)**2)

# --- Calcul variance de bloc ---
def theoretical_block_variance_block(geometry, size, resolution, sill, range_x, range_y, range_z, angle_deg, model_name):
    if resolution < 2:
        return sill, None, None, None

    x = np.linspace(0, size, resolution)
    cov_func = get_covariance_model(model_name)

    if geometry == 'ligne':
        dx = x[:, None] - x[None, :]
        h = anisotropic_distance(dx, 0, 0, range_x, range_y, range_z, angle_deg)
        cov = cov_func(h, range_=1.0, sill=sill)
        return np.mean(cov), x, None, None

    elif geometry == 'surface':
        X, Y = np.meshgrid(x, x)
        dx = X.ravel()[:, None] - X.ravel()[None, :]
        dy = Y.ravel()[:, None] - Y.ravel()[None, :]
        h = anisotropic_distance(dx, dy, 0, range_x, range_y, range_z, angle_deg)
        cov = cov_func(h, range_=1.0, sill=sill)
        return np.mean(cov), X, Y, None

    elif geometry == 'cube':
        X, Y, Z = np.meshgrid(x, x, x)
        dx = X.ravel()[:, None] - X.ravel()[None, :]
        dy = Y.ravel()[:, None] - Y.ravel()[None, :]
        dz = Z.ravel()[:, None] - Z.ravel()[None, :]
        h = anisotropic_distance(dx, dy, dz, range_x, range_y, range_z, angle_deg)
        cov = cov_func(h, range_=1.0, sill=sill)
        return np.mean(cov), X, Y, Z

# --- Interface utilisateur ---
geometry_input = widgets.Dropdown(options=['ligne', 'surface', 'cube'], value='surface', description='Géométrie')
block_size_input = widgets.FloatText(value=10, description='Taille bloc')
resolution_input = widgets.IntText(value=5, description='Résolution')
max_resolution_input = widgets.IntText(value=50, description='Résolution max')
sill_input = widgets.FloatText(value=1.0, description='Sill')
range_x_input = widgets.FloatText(value=30, description='Portée X')
range_y_input = widgets.FloatText(value=30, description='Portée Y')
range_z_input = widgets.FloatText(value=30, description='Portée Z')
angle_input = widgets.IntText(value=0, description='Angle (°)')
model_input = widgets.Dropdown(options=['sphérique', 'exponentiel', 'gaussien'], value='sphérique', description='Modèle')

calc_button = widgets.Button(description="Calculer", button_style='success')
output = widgets.Output()

def on_calc_clicked(b):
    with output:
        clear_output()
        
        geometry = geometry_input.value
        block_size = block_size_input.value
        resolution = resolution_input.value
        max_resolution = max_resolution_input.value
        sill = sill_input.value
        range_x = range_x_input.value
        range_y = range_y_input.value
        range_z = range_z_input.value
        angle_deg = angle_input.value
        model_name = model_input.value
        
        # Calcul variance pour plusieurs résolutions
        res_list = np.arange(2, max_resolution + 1)
        variances = []
        for res in res_list:
            var, _, _, _ = theoretical_block_variance_block(
                geometry, block_size, res,
                sill, range_x, range_y, range_z,
                angle_deg, model_name)
            variances.append(var)
        variances = np.array(variances)
        
        # Variance et discrétisation pour la résolution choisie
        var_current, X, Y, Z = theoretical_block_variance_block(
            geometry, block_size, resolution,
            sill, range_x, range_y, range_z,
            angle_deg, model_name)
        
        print(f"Variance de bloc estimée à résolution {resolution} : {var_current:.6f}")
        
        # Affichage graphique
        fig = plt.figure(figsize=(14,6))
        
        # Graphe variance vs résolution
        ax1 = fig.add_subplot(1, 2, 1)
        ax1.plot(res_list, variances, 'b-', label='Variance vs résolution')
        ax1.plot(resolution, var_current, 'ro', label=f'Résolution choisie = {resolution}')
        ax1.set_xlabel('Résolution (nombre de points)')
        ax1.set_ylabel('Variance de bloc estimée')
        ax1.set_title('Variance de bloc selon la résolution')
        ax1.grid(True)
        ax1.legend()
        
        # Discrétisation (points du bloc)
        ax2 = fig.add_subplot(1, 2, 2, projection='3d' if geometry=='cube' else None)
        
        if geometry == 'ligne':
            ax2.plot(X, np.zeros_like(X), 'o')
            ax2.set_xlim(0, block_size)
            ax2.set_ylim(-1, 1)
            ax2.set_title("Discrétisation de la ligne")
            ax2.set_xlabel("X")
        elif geometry == 'surface':
            ax2.plot(X, Y, 'ko')
            ax2.set_aspect('equal')
            ax2.set_title("Discrétisation de la surface")
            ax2.set_xlabel("X")
            ax2.set_ylabel("Y")
        else:  # cube
            ax2.scatter(X, Y, Z, c='k')
            ax2.set_title("Discrétisation du cube")
            ax2.set_xlabel("X")
            ax2.set_ylabel("Y")
            ax2.set_zlabel("Z")
        
        plt.tight_layout()
        plt.show()

calc_button.on_click(on_calc_clicked)

ui = widgets.VBox([
    geometry_input, block_size_input, resolution_input, max_resolution_input,
    sill_input, range_x_input, range_y_input, range_z_input,
    angle_input, model_input,
    calc_button,
    output
])

display(ui)

